In [1]:
!python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 827.9 MB 1.5 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=d83772f19e3d564648520d2015a020b93f7c515e1a3d2aeda299ca5ce5f0d047
  Stored in directory: /tmp/pip-ephem-wheel-cache-451rc65u/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
!pip3 install git+https://github.com/laxmimerit/preprocess_kgptalkie.git

In [2]:
import spacy
import pickle
import sklearn
import en_core_web_lg
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.tokenize import RegexpTokenizer
import string
import re
english_punctuations = string.punctuation
punctuations_list = english_punctuations
# STOP WORDS
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
#remove stopwords 
STOPWORDS = set(stopwords.words('english'))
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
import seaborn as sns


number_to_sentiment = {
    0 : "sadness",
    1 : "worry",
    2 : "anger",
    3 : "neutral",
    4 : "happiness",
    5 : "love"
}

In [3]:
nlp = en_core_web_lg.load()

### Model Building

In [7]:
df = pd.read_csv('../data/cleaned_tweet_emotions.csv')

In [8]:
df.head()

,content,sentiment
1,"['layin', 'n', 'bed', 'headache', 'ughhhhwaiti...",0
2,"['funeral', 'ceremonygloomy', 'friday']",0
3,"['wants', 'hang', 'friends', 'soon']",4
4,"['dannycastillo', 'we', 'want', 'trade', 'some...",3
5,"['repinging', 'ghostridah14', 'didnt', 'go', '...",1


### `word2vec` Conversion

In [17]:
def get_vec(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

In [20]:
df['vec'] = df['content'].apply(lambda x: get_vec(x))

In [21]:
df.head()

,content,sentiment,vec
1,"['layin', 'n', 'bed', 'headache', 'ughhhhwaiti...",0,"[-0.15106119, 0.3921255, -0.023442717, -0.1402..."
2,"['funeral', 'ceremonygloomy', 'friday']",0,"[-0.17736591, 0.42403764, 0.0023445403, -0.091..."
3,"['wants', 'hang', 'friends', 'soon']",4,"[-0.12679328, 0.3603656, -0.05800653, -0.09750..."
4,"['dannycastillo', 'we', 'want', 'trade', 'some...",3,"[-0.18136817, 0.45698026, -0.012877913, -0.085..."
5,"['repinging', 'ghostridah14', 'didnt', 'go', '...",1,"[-0.15367743, 0.3966674, -0.040145695, -0.1753..."


In [22]:
X = df['vec'].to_numpy()
X = X.reshape(-1, 1)

In [23]:
X.shape

(66679, 1)

In [24]:
X = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(-1, 300)

In [25]:
X.shape

(66679, 300)

In [27]:
y = df['sentiment']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

In [29]:
X_train.shape, X_test.shape

((53343, 300), (13336, 300))

### Train ML Model

In [30]:
clf = LogisticRegression(solver='liblinear')

In [31]:
clf.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [32]:
y_pred = clf.predict(X_test)

In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.48      0.15      0.23      1270
           1       0.41      0.27      0.32      1905
           2       0.50      0.22      0.31      1333
           3       0.50      0.79      0.61      4822
           4       0.52      0.49      0.51      2835
           5       0.61      0.40      0.49      1171

    accuracy                           0.50     13336
   macro avg       0.50      0.39      0.41     13336
weighted avg       0.50      0.50      0.47     13336



In [35]:
pickle.dump(clf, open('w2v_sentiment.pkl', 'wb'))

In [36]:
model = pickle.load(open('w2v_sentiment.pkl', 'rb'))

In [49]:
def cleaning_tweet(text):
    # removing punctuation
    translator = str.maketrans('', '', punctuations_list)
    text = text.translate(translator)

    # removing stop words 
    text = " ".join([word for word in str(text).split() if word not in STOPWORDS])

    # removing pseudos
    text = text.replace('@[a-zA-Z0-9-_]*',"")

    # to lower case
    text = text.lower()

    # remove URL
    text = re.sub('http\S+',"", text)

    # tokenization
    tokenizer = RegexpTokenizer(r'\w+|$[0-9]+|\S+')
    tokenizer.tokenize(text)

    return text

In [42]:
x = 'this was really a $%good movie'

x = cleaning_tweet(x)
vec = get_vec(x)
print(vec)

y = model.predict(vec.reshape(1, -1))
number_to_sentiment[y[0]]

array([-1.43590003e-01,  1.58450007e-01, -1.82820007e-01, -2.27396667e-01,
        1.62013009e-01,  2.90702671e-01,  1.18126333e-01, -4.76989985e-01,
        1.35988012e-01,  2.23903346e+00, -2.32823327e-01,  1.03273332e-01,
        2.31916323e-01,  2.05216650e-02, -2.78753310e-01, -1.53190657e-01,
       -5.28000807e-03,  8.67619991e-01, -4.17683333e-01,  8.35463330e-02,
       -8.46000016e-02, -2.37386703e-01,  3.66799980e-01, -1.56795993e-01,
       -4.06316631e-02,  3.26756686e-01, -6.20186664e-02, -3.43816668e-01,
       -4.53400016e-02, -2.78986663e-01, -5.21303304e-02, -1.28143325e-01,
       -6.29486665e-02,  1.32602334e-01,  5.95133305e-02, -3.54660004e-02,
        2.27786660e-01,  4.22689952e-02, -3.90443325e-01, -6.49366602e-02,
        1.33999661e-01,  2.75566634e-02, -2.86864694e-02, -1.77450001e-01,
        3.13410014e-01,  2.04822659e-01, -1.15996659e-01,  3.58356684e-02,
       -1.66852340e-01, -2.24059984e-01,  9.54273269e-02,  1.42066658e-01,
        2.41903350e-01, -

In [ ]:
def plot_confusion_matrix(y_test, y_pred):
    labels = unique_labels(y_test)
    column = [f'Predicted {number_to_sentiment[label]}' for label in labels]
    indices = [f'Actual {number_to_sentiment[label]}' for label in labels]
    table = pd.DataFrame(confusion_matrix(y_test, y_pred), columns = column, index = indices)

    plt.figure(figsize=(10,10))
    sns.heatmap(table, annot=True)
    plt.title('Confusion Matrix')
    plt.ylabel('Actal Values')
    plt.xlabel('Predicted Values')
    plt.show()

In [ ]:
plot_confusion_matrix(y_test, y_pred)